# Single point analysis of least-cost paths

Here we pick a single point, somewhere in the world, and find the least-cost path to all other points.
It is then visualised, so we can ensure things are making sense.

Most of the time, we'd want to just get the cost of all paths to a point, using the `get_dist_from_point` function, but here we just look at how the algorithm works for a single point.

In [ ]:
import meshio
import numpy as np
from multiprocessing import Pool
import time

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
import matplotlib.tri as mtri

label_size = 8
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

from gLEC.gLEC import gLEC

# Load the mesh

We need to load in the mesh. We can visualise it with matplotlib:

In [ ]:
infile = "output_data/AUS_LR.vtk"

mesh = meshio.read(infile)

triang = mtri.Triangulation(mesh.points[:,0], mesh.points[:,1], mesh.cells[0].data)
fig, ax = plt.subplots()
ax.triplot(triang, zorder=1)
ax.set_aspect('equal')

# Create a gLEC object

Here we create a gLEC object with the mesh.

It's not doing very much, so let's start using it.

In [ ]:
lec_calculator = gLEC(mesh)

# Pick a starting point

To do a single point analysis, we need to start with a single point. 

We don't need a particular point, so we could do something like this:
```python
starting_point = 0
```
which would use the first point defined in the mesh. However, for this example, we want to use a slightly nicer point, so we use this simple function to find a better one:

In [ ]:
starting_point = mesh.points.shape[0]//9 * 9
print(f'Starting point index: {starting_point}')
print(f'Starting point value:\n{mesh.points[starting_point,:]}')

# Find a nearby neighbour

Many of the functions of the `gLEC` rely on comparing between two connected points. We can use `gLEC` to tell us which points are connected to our `starting_point`:

In [ ]:
nearby_points = lec_calculator.neighbours_func(starting_point)
print(f'Nearby points indexs: {nearby_points}')
print(f'Nearby points values:\n{mesh.points[nearby_points,:]}')

## Showing the points

We can show where these points are on our mesh.

*NOTE* The code below is not important for using `gLEC`, but is a nice way to show the points.

In [ ]:
fig, ax = plt.subplots()

ax.triplot(triang, zorder=1)

ax.scatter(mesh.points[starting_point,0],
           mesh.points[starting_point,1],
           c='yellow', s =20, zorder=3)

ax.scatter(mesh.points[nearby_points,0],
           mesh.points[nearby_points,1],
           c='orange', s =20, zorder=2)

ax.set_aspect('equal')

axins = zoomed_inset_axes(ax, 15, loc=2)
axins.triplot(triang, zorder=1)

axins.scatter(mesh.points[starting_point,0],
           mesh.points[starting_point,1],
           c='yellow', s =20, zorder=3)

axins.scatter(mesh.points[nearby_points,0],
           mesh.points[nearby_points,1],
           c='orange', s =20, zorder=2)

zoom_min = np.min(mesh.points[nearby_points,:], axis=0)
zoom_max = np.max(mesh.points[nearby_points,:], axis=0)
span = zoom_max - zoom_min
zoom_min -= span * 0.5
zoom_max += span * 0.5

axins.set_xlim(zoom_min[0], zoom_max[0])
axins.set_ylim(zoom_min[1], zoom_max[1])
axins.xaxis.set_visible('False')
axins.yaxis.set_visible('False')
_ = mark_inset(ax, axins, loc1=1, loc2=4, fc="none", ec="0.5")

## Picking one of the nearby points

We only need one nearby point, so we'll pick the first one

In [ ]:
nearby_point = nearby_points[0]
print(f'Nearby point index: {nearby_point}')
print(f'Nearby point value:\n{mesh.points[nearby_point,:]}')

## Elevation data

Notice that in this case, the `Z` value has been 0 for out starting and nearby points. This is because the elevation data is stored on additional mesh variable:

In [ ]:
for point in (starting_point, nearby_point):
    print(f"Elevation of {point = } is {mesh.point_data['Z'][point]} m")

# Using the gLEC calculator for comparisions

The `gLEC` object has some built in calculators to help us make comparisions between points:

In [ ]:
dist = lec_calculator.dist_func(starting_point, nearby_point)
print(f"Distance between {starting_point=} and {nearby_point=} is\n{dist} m\n")

cost = lec_calculator.travel_cost_func(starting_point, nearby_point)
print(f"The travel cost between {starting_point=} and {nearby_point=} is\n{cost} units of fuel")

### Travel cost and fuel

The distance function (`dist_func(starting_point, nearby_point)`) is a simple euclidean distance between the two points.

### Change the functions

We can override the default functions of the LECMesh when we create it:

In [ ]:
# Here is a new travel cost function, and it *only* uses elevation as a cost measure.
def elevation_only(current, _next):
    # Only take into account elevation changes for costs
    if current == _next:
        return 0
    return int(abs(mesh.point_data['Z'][current] - mesh.point_data['Z'][_next]))

In [ ]:
lm_ele = LECMesh(mesh, max_fuel = 2500, travel_cost_function = elevation_only)

In [ ]:
lm_ele.travel_cost_function(starting_point, near_point)  # Test the travel_cost function between two points

In [ ]:
lm_ele.dist_func(starting_point, near_point)  # Test the distance function between two points

Note that the distance is the same, but the cost is diffent now, since the cost is now *only* the change in elevation:

In [ ]:
print("Starting point elevation: ", mesh.point_data['Z'][starting_point])
print("Nearby point elevation:   ", mesh.point_data['Z'][near_point])
print("Change in elevation:      ", mesh.point_data['Z'][starting_point] - mesh.point_data['Z'][near_point])

# Computing the least-cost paths

Going back to our original `lm` LECMesh object, we can call the `cost_search` function on the `starting_point`, which shows all the paths starting at that point

In [ ]:
# Do the least-cost path calculations, and get back the data
#came_from, cost_so_far = cost_search(mesh, starting_point, travel_cost_function = travel_cost, max_distance = max_distance)
came_from, cost_so_far, dist_so_far = lm.cost_search(starting_point)

Now let's see the data structures that come out of that function

### came_from
`came_from` is a dictionary, where the keys are point IDs, and the value is the point ID of the point it came_from.

In [ ]:
# Show some of the came_from data structure
count = 0
num_points_to_show = 80
for point, preceeding_point in came_from.items():
    print(str(point) + "\tcame from\t" + str(preceeding_point))
    if count > num_points_to_show:
        break
    count +=1

You can see above that point `66772` "came_from" `None`, since it was the initial point. 

This also means you can pick a point, and follow it back until you reach the starting point, AKA, when the `came_from` value is `None`.
In this case, we'll pick a point from the end of the list of points using: `list(came_from.keys())[-1]`

In [ ]:
point = came_from[list(came_from.keys())[-1]]  # choose a start point

In [ ]:
# follow a single path back to the origin point
current_point = point
while current_point:   # while point is not None
    print(str(current_point), end="")
    
    # Update the current point to follow the came_from
    current_point = came_from[current_point]
    
    if current_point:
        print(" -> ", end="")

### cost_so_far

`cost_so_far` is also a dictionary, where the keys are point IDs, and the value is total cost it has taken to reach that point, from the starting point

In [ ]:
# the cost to get from the starting point to the point we picked
print(cost_so_far[point])

In [ ]:
# Show the progressive cost of going from a point to the starting point
current_point = point
while current_point:   # while point is not None
    print(str(cost_so_far[current_point]), end="")
    current_point = came_from[current_point]
    if current_point:
        print(" -> ", end="")

### dist_so_far

`dist_so_far` is the same structure as `cost_so_far`, but it shows the length of a path up to that point.

In [ ]:
print(dist_so_far[point])

In [ ]:
# Show the progressive distance of going from a point to the starting point
current_point = point
while current_point:   # while point is not None
    print(str(dist_so_far[current_point]), end="")
    current_point = came_from[current_point]
    if current_point:
        print(" -> ", end="")

We can also find the longest path possible with the given max_fuel

In [ ]:
print("Longest path: {:.3f} km".format(max(dist_so_far.values())/1000))

## Show a map of all shortest paths

Since all paths have to lead back to the starting point, we can map out each of their paths by finding edge nodes, and then following the `came_from` paths back to the starting point.

In [ ]:
# Find all the nodes that are at the edge of the tree
edge_nodes = []

for k in came_from.keys():             # For all the points we've visited,
    if k not in came_from.values():    # Find all the points that haven't been 'came_from'
        edge_nodes.append(k)
        
print(edge_nodes)

In [ ]:
# For each edge node, follow the path back to the starting point, and keep track of the points and costs along the way
paths = []
costs = []
dists = []
for p in edge_nodes:
    point = p
    cost = 0
    new_points = []
    new_costs = []
    while point:
        new_points.append(mesh.points[point])  # note, the points are being pulled from the VTK, so we get all their info
        new_costs.append(cost_so_far[point])
        point = came_from[point]

    new_points = np.array(new_points)
    new_costs  = np.array(new_costs)
    paths.append(new_points)
    costs.append(new_costs)

In [ ]:
total_dist = 0
for k in came_from.keys():             # For all the points we've visited,
    if k not in came_from.values():    # Find all the points that haven't been 'came_from'
        total_dist += dist_so_far[k]
print(total_dist)

In [ ]:
# Visualise all the paths back to the starting point, with colours showing the cost along the way

%matplotlib notebook
import matplotlib.pyplot as plt

norm = plt.Normalize(0, max_cost)
for p, c in zip(paths, costs):    
    plt.plot(p[:,0], p[:,1], c='k', zorder=0)
    plt.scatter(p[:,0], p[:,1], c=c, norm=norm, zorder=1)
    
plt.colorbar()

In [ ]:
# Visualise all the paths back to the starting point, with colours showing the cost along the way
# In 3D - depending on where you pick on the globae, the 2D can come out all squished. 3D avoids this.
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

norm = plt.Normalize(0, max_cost)
for p, c in zip(paths, costs):    
    ax.plot(p[:,0], p[:,1], p[:,2], c='k', zorder=0)
    ax.scatter(p[:,0], p[:,1], p[:,2], c=c, norm=norm, zorder=1)
    
#plt.colorbar()
#plt.axes().set_aspect('equal', 'datalim')